In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#  Load your dataset
df = pd.read_csv('Error Annotated Corpus.csv')  # Replace with your dataset path

# Drop rows with missing values
df = df.dropna(subset=['Error word & consecutive word', 'Corrected words & its'])

# Preprocess the text (strip extra spaces)
df['Error word & consecutive word'] = df['Error word & consecutive word'].apply(lambda x: x.strip())
df['Corrected words & its'] = df['Corrected words & its'].apply(lambda x: x.strip())

# Example of what the dataset might look like
print(df.head())


  Error word & consecutive word     Corrected words & its    Annotation
0               10கனநீர் உலைகள்           10கணநீர் உலைகள்  வேற்றெழுத்து
1                   அகளவிரித்து               அகலவிரித்து  வேற்றெழுத்து
2      அணைத்து ஊழியர்களுக்குமான  அனைத்து ஊழியர்களுக்குமான  வேற்றெழுத்து
3               அதள பாதாளதுக்கு         அதல பாதாளத்துக்கு  வேற்றெழுத்து
4           அதற்குறிய தீர்வுகளை       அதற்குரிய தீர்வுகளை  வேற்றெழுத்து


In [2]:
df = df.drop(columns=['Annotation'])


In [3]:
print(df.head())

  Error word & consecutive word     Corrected words & its
0               10கனநீர் உலைகள்           10கணநீர் உலைகள்
1                   அகளவிரித்து               அகலவிரித்து
2      அணைத்து ஊழியர்களுக்குமான  அனைத்து ஊழியர்களுக்குமான
3               அதள பாதாளதுக்கு         அதல பாதாளத்துக்கு
4           அதற்குறிய தீர்வுகளை       அதற்குரிய தீர்வுகளை


In [4]:
# Drop rows with missing values (use actual column names)
df = df.dropna(subset=['Error word & consecutive word', 'Corrected words & its'])

In [5]:
# Character-level n-grams extraction with reduced n (2-grams)
def extract_ngrams(text, n=2):
    return [text[i:i+n] for i in range(len(text) - n + 1)]

In [6]:
# Convert the 'Error word & consecutive word' into character n-grams
df['Error word ngrams'] = df['Error word & consecutive word'].apply(lambda x: ' '.join(extract_ngrams(x, n=2)))  # Using 2-grams

# Prepare the features and labels
X = df['Error word ngrams']  # Features: character n-grams from the incorrect words
y = df['Corrected words & its']  # Labels: the corrected words

In [7]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# Use CountVectorizer to convert the n-grams into numerical features (limiting to 10,000 features)
vectorizer = CountVectorizer(analyzer=lambda x: x.split(), max_features=10000)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# Train a Logistic Regression Classifier (less memory-intensive than Random Forest)
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_vect, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [22]:
# Evaluate the model
y_pred = model.predict(X_test_vect)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 43.25%


In [11]:
# Function to correct spelling in a paragraph
def correct_paragraph(paragraph):
    words = paragraph.split()  # Split the paragraph into words
    corrected_words = []

    for word in words:
        # Extract n-grams for the word
        ngrams = ' '.join(extract_ngrams(word))

        # Transform the n-grams to features using the same vectorizer
        word_vect = vectorizer.transform([ngrams])

        # Predict the corrected word using the trained model
        corrected_word = model.predict(word_vect)[0]

        # Append the corrected word
        corrected_words.append(corrected_word)

    # Join the corrected words back into a sentence
    corrected_paragraph = ' '.join(corrected_words)
    return corrected_paragraph


In [16]:
# Test with a sample paragraph
input_paragraph = " அதற்குறிய தீர்வுகளை செலவேன்"
corrected_paragraph = correct_paragraph(input_paragraph)
print(f"Original: {input_paragraph}")
print(f"Corrected: {corrected_paragraph}")

Original:  அதற்குறிய தீர்வுகளை செலவேன்
Corrected: அதற்கு அதற்குரிய தீர்வுகளை செய்தேன்


In [19]:
df.head(50)

,Error word & consecutive word,Corrected words & its,Error word ngrams
0,10கனநீர் உலைகள்,10கணநீர் உலைகள்,10 0க கன னந நீ ீர ர் ் உ உல லை ைக கள ள்
1,அகளவிரித்து,அகலவிரித்து,அக கள ளவ வி ிர ரி ித த் ்த து
2,அணைத்து ஊழியர்களுக்குமான,அனைத்து ஊழியர்களுக்குமான,அண ணை ைத த் ்த து ு ஊ ஊழ ழி ிய யர ர் ்க கள ள...
3,அதள பாதாளதுக்கு,அதல பாதாளத்துக்கு,அத தள ள ப பா ாத தா ாள ளத து ுக க் ்க கு
4,அதற்குறிய தீர்வுகளை,அதற்குரிய தீர்வுகளை,அத தற ற் ்க கு ுற றி ிய ய த தீ ீர ர் ்வ வு ு...
5,அதற்கேற்றார்போல அதன்,அதற்கேற்றாற்போல அதன்,அத தற ற் ்க கே ேற ற் ்ற றா ார ர் ்ப போ ோல ல ...
6,அந்நியமாக நடந்தது,"அன்னியமாக,அன்னியமாக்க நடந்தது",அந ந் ்ந நி ிய யம மா ாக க ந நட டந ந் ்த தத து
7,அம்பிலாந்துறை,அம்பிளாந்துறை,அம ம் ்ப பி ில லா ாந ந் ்த து ுற றை
8,அரவனைத்துச் செல்கின்ற,அரவணைத்துச் செல்கின்ற,அர ரவ வன னை ைத த் ்த து ுச ச் ் ச செ ெல ல் ்...
9,அருவறுப்பாக நினைக்கிறார்கள்,அறுவறுப்பாக நினைக்கிறார்கள்,அர ரு ுவ வற று ுப ப் ்ப பா ாக க ந நி ின னை ை...
